In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup # For web scraping

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [266]:
cafe_df = pd.read_csv('cafe_addresses.csv')
cafe_df.head(10)

,Name,Address,City,PostalCode,Country,Latitude,Longitude
0,Brot & Spiele,Mariahilferstraße 17,Graz,8020,Austria,47.073272,15.433036
1,Brot und Spiele,Laudongasse 22,Vienna,1080,Austria,48.213407,16.349799
2,Café Benno,Alser Str. 67,Vienna,1080,Austria,48.215050,16.342587
3,Café Sperlhof,Große Sperlgasse 41,Vienna,1020,Austria,48.219658,16.378380
4,SpielBar,Lederergasse 26,Vienna,1080,Austria,48.213688,16.348476
5,The Playground (Hoofdkerk),Hoofdkerkstraat 7,Antwerp,2000,Belgium,51.221243,4.403738
6,The Playground (Station),Pelikaanstraat 3/1270,Antwerp,2018,Belgium,51.216644,4.420792
7,Outpost Antwerpen,Beggaardenstraat 6,Antwerp,2000,Belgium,51.219510,4.404468
8,La Luck Brussels,74 rue Washington,Brussels,1050,Belgium,50.823495,4.363401
9,La Table Food & Games,63 rue de l'enseignement,Brussels,1000,Belgium,50.848845,4.365281


In [305]:
city_list = []
for n in range(1, 6):
    url = f'http://www.citymayors.com/features/euro_cities{n}.html'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    table = soup.find_all('table')[1]
    city_table = [td.text.title().rstrip() for td in table.find_all('td', attrs={'width':'140'})[1:]]
    city_list.extend(city_table)

Unfortunately, the data on City Mayors omits Turkey, and so we need to collect this information from [Wikipedia](https://en.wikipedia.org/wiki/List_of_largest_cities_and_towns_in_Turkey). The smallest population value given in the City Mayors table is 149,000 people, and so we will include all cities whose population exceed this value.

In addition, a number of English counties, rather than cities are within the data, so we shall remove those, and we need to include 'Rome' instead of 'Roma' to ensure it is properly treated.

In [306]:
city_list = list(map(lambda x: x.replace('Roma', 'Rome'), city_list))

turkey_url = 'https://en.wikipedia.org/wiki/List_of_largest_cities_and_towns_in_Turkey'
turkey_resp = requests.get(turkey_url)
turkey_soup = BeautifulSoup(turkey_resp.content, 'lxml')
turkey_table = turkey_soup.find('table', attrs={'class': 'sortable'})
city_pop = [td.text.rstrip('\n').replace(',', '') for td in turkey_table.find_all('td')[6::8]]
turkey_cities = [a.text for a in turkey_table.find_all('a')[::2]]

In [307]:
for city, pop in zip(turkey_cities, city_pop):
    try:
        if int(pop) > 149000:
            city_list.append(city)
    except:
        pass
print(f'city_list now contains {len(city_list)} cities')

city_list now contains 543 cities


In [308]:
for city in city_list:
    if 'shire' in city:
        city_list.remove(city)
print(f'city_list now contains {len(city_list)} cities')

city_list now contains 534 cities


In [309]:
cafe_city_list = cafe_df['City'].unique().tolist()
cafe_city_list.sort()
#cafe_city_list

In [310]:
city_list.sort()
#city_list

In [311]:
city_without_cafe = []

In [293]:
def has_a_cafe(city):
    inclusion = [(cafe.lower() in city.lower()) for cafe in cafe_city_list]
    return any(inclusion)

In [312]:
for city in city_list:
    if not has_a_cafe(city):
        city_without_cafe.append(city)
print(f'\ncity_without_cafe has {len(city_without_cafe)} cities')


city_without_cafe has 439 cities


In [313]:
city_without_cafe

['Aachen',
 'Abakan',
 'Aberdeen',
 'Adana',
 'Adapazarı',
 'Adıyaman',
 'Afyon',
 'Aksaray',
 'Alcalá De Henares',
 'Alicante',
 'Almería',
 'Angarsk',
 'Antakya',
 'Antalya',
 'Apeldoorn',
 'Arad',
 'Arkhangelsk',
 'Armavir',
 'Astrakhan',
 'Augsburg',
 'Aydın',
 'Aylesbury Vale',
 'Bacau',
 'Badalona',
 'Baia Mare',
 'Balakovo',
 'Balıkesir',
 'Banja Luka',
 'Baranovichi',
 'Bari',
 'Barnaul',
 'Barnsley',
 'Basildon',
 'Batman',
 'Belaya Tserkov',
 'Belfast',
 'Belgorod',
 'Beltsy',
 'Berezniki',
 'Bergen',
 'Bialystok',
 'Bielefeld',
 'Bielsko-Biala',
 'Biisk',
 'Bilbao',
 'Blackpool',
 'Blagoveshchensk',
 'Bobruisk',
 'Bologna',
 'Borisov',
 'Bourgas',
 'Bournemouth',
 'Braila',
 'Brasov',
 'Bratislava',
 'Bratsk',
 'Braunschweig',
 'Bremen',
 'Brescia',
 'Brest',
 'Bryansk',
 'Burgos',
 'Bursa',
 'Bury',
 'Bydgoszcz',
 'Bytom',
 'Bâle',
 'Caerphilly',
 'Cagliari',
 'Calderdale',
 'Cartagena',
 'Catania',
 'Charleroi',
 'Charnwood',
 'Cheboksary',
 'Chelyabinsk',
 'Chemnitz',
 'C